In [1]:
from service.rag_processing import TextHandlingService
from langchain_openai import OpenAIEmbeddings  
from pymongo import MongoClient
from langchain_openai import ChatOpenAI
from langchain_mongodb import MongoDBAtlasVectorSearch
import os
from langgraph.graph import MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
import uuid
from langchain_core.messages import SystemMessage
from langchain_mongodb import MongoDBChatMessageHistory
from service.element import Element
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_core.messages import AIMessage
from slack_sdk import WebClient

self_service = TextHandlingService()

client = MongoClient(os.environ["MONGODB_ATLAS_CLUSTER_URI"])
db = "text"
collection_name = "text"
search_index = "hello"
collection = client[db][collection_name]

embedder = OpenAIEmbeddings(
        model="azure-text-embedding-3-large",
        api_key=os.environ["OPENAI_API_KEY"],
        base_url=os.environ['AZURE_OPENAI_ENDPOINT']
    )

model = ChatOpenAI(
        base_url=os.environ["AZURE_OPENAI_ENDPOINT"],
        model=os.environ["AZURE_OPENAI_COMP_DEPLOYMENT_NAME"],
        api_key=os.environ["OPENAI_API_KEY"]
    )

vectorstore = MongoDBAtlasVectorSearch(
            collection=collection,
            embedding=embedder,  # same embedding model used when saving
            index_name=search_index,
            relevance_score_fn="cosine"
        )

text = "Hello, My name is Halo."

values = self_service.splitting(text)

vector_store = self_service.embed_documents(values, embedder, collection, search_index)

